This file uses floweaver to generate Sankey diagrams that map coarse officers to appointments.

Below is the instruction to run the file.

### Install floweaver in Anaconda Prompt (if use Jupyter Notebook) / PyCharm Terminal:

pip install floweaver

pip install ipysankeywidget

jupyter nbextension enable --py --sys-prefix ipysankeywidget

jupyter notebook (to open jupyter notebook)

### To display and save the output figures:
Select Start Jupyter Server from the Jupyter Actions Menu (lightbulb icon next to Run All cells icon)

Open Event Log

Open in Browser

Find the script and run all cells

In [1]:
import tlo

import pandas as pd
import numpy as np
from ipysankeywidget import SankeyWidget
from matplotlib import pyplot as plt
from floweaver import *
from pathlib import Path
from ipywidgets import HBox, VBox

# get the tlo path
tlopath = Path(tlo.__file__).parent.parent.parent

# Get the path of current folder that stores the data
workingpath = tlopath / Path('resources/healthsystem/human_resources/definitions')

# Define the path of output Sankeys
outputpath = tlopath / Path('outputs/healthsystem/human_resources/sankey_diagrams')

# Read the data of appointment time table
appointment = pd.read_csv(workingpath / 'ResourceFile_Appt_Time_Table.csv')

# Rename
appointment.loc[appointment['Officer_Category'] == 'Nursing_and_Midwifery',
                'Officer_Category'] = 'Nursing and Midwifery'

# Read the data of appointment types table
appt_types = pd.read_csv(workingpath / 'ResourceFile_Appt_Types_Table.csv')
# Rename
appt_types.loc[appt_types['Appt_Cat'] == 'GENERAL_INPATIENT_AND_OUTPATIENT_CARE',
              'Appt_Cat'] = 'IPOP'
appt_types.loc[appt_types['Appt_Cat'] == 'Nutrition',
              'Appt_Cat'] = 'NUTRITION'
appt_types.loc[appt_types['Appt_Cat'] == 'Misc',
              'Appt_Cat'] = 'MISC'
appt_types.loc[appt_types['Appt_Cat'] == 'Mental_Health',
              'Appt_Cat'] = 'MENTAL'

# Merge appt category to the time table
appointment = appointment.merge(appt_types[['Appt_Type_Code', 'Appt_Cat']],
                               on='Appt_Type_Code', how='left')

# Add prefix 'Facility_Level'
appointment['Facility_Level'] = 'Facility_Level_' + appointment['Facility_Level'].astype(str)

# Draw a diagram using appointment time table itself.
# Currentlt, we do not know how many appointments of a type at a level happened or \
# how many minutes of an officer category at a level go to that appointment.
# We consider the mapping between officer categories and appointment types, \
# which can be derived from the appointment time table.

In [ ]:
# The flow maps 9 officer categories and 11 appt cateogories at all levels
flow_coarse_officer_appt = pd.DataFrame(
    appointment.groupby(['Officer_Category', 'Appt_Cat', 'Facility_Level'],
                        dropna=False, sort=False).sum()
).reset_index()
# Drop column of minutes and add column 'value'
flow_coarse_officer_appt.drop(columns = ['Time_Taken_Mins'], inplace=True)
flow_coarse_officer_appt['value'] = 1

# Add 'source' and 'target' columns
flow_coarse_officer_appt['source'] = 'Officer_Category'
flow_coarse_officer_appt['target'] = 'Appt_Cat'

size = dict(width=800, height=800, margins=dict(left=180, right=180))

# Different ways to order nodes
# Sorted
# partition_officer_cat = Partition.Simple('Officer_Category',
#                                              np.unique(flow_coarse_officer_appt['Officer_Category']))
# Unsorted
# partition_officer_cat = Partition.Simple('Officer_Category',
#                                              pd.unique(pd.Series(flow_coarse_officer_appt['Officer_Category'])))
# Fixed
partition_officer_cat = Partition.Simple('Officer_Category',
                                         pd.array(['DCSA', 'Clinical', 'Nursing and Midwifery', 'Pharmacy',
                                                   'Laboratory', 'Dental', 'Radiography', 'Mental']))

# partition_appt_cat = Partition.Simple('Appt_Cat',
#                                           np.unique(flow_coarse_officer_appt['Appt_Cat']))
partition_appt_cat = Partition.Simple('Appt_Cat',
                                      pd.array(['ConWithDCSA', 'IPOP', 'RMNCH', 'MISC',
                                      'HIV', 'TB', 'NUTRITION', 'PharmDispensing', 'LABORATORY',
                                      'DENTAL', 'RADIOGRAPHY', 'MENTAL']))

partition_facility_level = Partition.Simple('Facility_Level',
                                            np.unique(flow_coarse_officer_appt['Facility_Level']))

nodes = {
    'Officer': ProcessGroup(['Officer_Category'], partition_officer_cat),
    'Appt': ProcessGroup(['Appt_Cat'], partition_appt_cat),
}

# Add nodes Waypoint
nodes['waypoint'] = Waypoint(partition_facility_level)

bundles = [
    Bundle('Officer', 'Appt', waypoints=['waypoint']),
]

ordering = [
    ['Officer'],  # left
    ['waypoint'],    # middle
    ['Appt'],     # right
    ]

# Set the color for each officer category
palette = {'Clinical': 'skyblue', 'Nursing and Midwifery': 'lightpink',
           'Pharmacy': 'khaki', 'Laboratory': 'cadetblue',
           'Radiography': 'yellowgreen', 'Dental': 'salmon',
           'Mental': 'mediumorchid', 'DCSA': 'royalblue'
          }


# Sankey diagram definition (SDD)
sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_officer_cat)

sankey_coarse_officer_and_coarse_appt = weave(sdd, flow_coarse_officer_appt,
                                                palette=palette, measures='value').to_widget(**size)

sankey_coarse_officer_and_coarse_appt.auto_save_png(outputpath /'Sankey_coarse_officer_and_coarse_appt.png')

In [ ]:
# The flow maps 9 officer categories and 51 appt types at an individaul level
flow_coarse_officer = pd.DataFrame(
    appointment.groupby(['Officer_Category', 'Appt_Type_Code', 'Facility_Level'],
                        dropna=False, sort=False).sum()
).reset_index()
# As we do not care about the flow proportions, we add a 'value' columns with constant value 1.
flow_coarse_officer['value'] = 1
# Add 'source' and 'target' columns
flow_coarse_officer['source'] = 'Officer_Category'
flow_coarse_officer['target'] = 'Appt_Type_Code'

def sankey_level_coarse_officer(level, h):
    flow_coarse_officer_level = flow_coarse_officer.loc[flow_coarse_officer['Facility_Level'] == level, :].copy()
    flow_coarse_officer_level.drop(columns = 'Facility_Level', inplace=True)
    flow_coarse_officer_level.reset_index(drop=True, inplace=True)

    size = dict(width=800, height=h, margins=dict(left=180, right=180))

    partition_officer_cat = Partition.Simple('Officer_Category',
                                             pd.array(['DCSA', 'Clinical', 'Nursing and Midwifery', 'Pharmacy',
                                                       'Laboratory', 'Radiography', 'Dental', 'Mental']))
    partition_appt_type = Partition.Simple('Appt_Type_Code', pd.unique(pd.Series(appt_types['Appt_Type_Code'])))


    nodes = {
        'Officer': ProcessGroup(['Officer_Category'], partition_officer_cat),
        'Appt': ProcessGroup(['Appt_Type_Code'], partition_appt_type),
    }

    bundles = [
        Bundle('Officer', 'Appt'),
    ]

    ordering = [
        ['Officer'],
        ['Appt'],
    ]

    # Set the color for each officer category
    palette = {'Clinical': 'skyblue', 'Nursing and Midwifery': 'lightpink',
               'Pharmacy': 'khaki', 'Laboratory': 'cadetblue',
               'Radiography': 'yellowgreen', 'Dental': 'salmon',
               'Mental': 'mediumorchid', 'DCSA': 'royalblue'
              }

    sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition=partition_officer_cat) # color by officer cat

    return weave(sdd, flow_coarse_officer_level, palette=palette, measures='value').to_widget(**size)

sankey_coarse_officer_level_0 = sankey_level_coarse_officer('Facility_Level_0', 100)
sankey_coarse_officer_level_0.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_0.png')

sankey_coarse_officer_level_1a = sankey_level_coarse_officer('Facility_Level_1a', 1200)
sankey_coarse_officer_level_1a.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_1a.png')

sankey_coarse_officer_level_1b = sankey_level_coarse_officer('Facility_Level_1b', 1200)
sankey_coarse_officer_level_1b.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_1b.png')

sankey_coarse_officer_level_2 = sankey_level_coarse_officer('Facility_Level_2', 1200)
sankey_coarse_officer_level_2.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_2.png')

sankey_coarse_officer_level_3 = sankey_level_coarse_officer('Facility_Level_3', 1200)
sankey_coarse_officer_level_3.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_3.png')

sankey_coarse_officer_level_4 = sankey_level_coarse_officer('Facility_Level_4', 200)
sankey_coarse_officer_level_4.auto_save_png(outputpath /'Sankey_coarse_officer_and_fine_appt_level_4.png')

top_box = HBox([sankey_coarse_officer_level_0, sankey_coarse_officer_level_4])
mid_box = HBox([sankey_coarse_officer_level_1a, sankey_coarse_officer_level_1b])
bottom_box = HBox([sankey_coarse_officer_level_2, sankey_coarse_officer_level_3])
VBox([top_box, mid_box, bottom_box])